In [23]:
import h5py
import numpy as np
import os
import torch

In [26]:
perc = 125
it = 2
conv_type = 'gcn'
input_size = 7 # dataset features: Time, Press_SA, Press_abd, FlowRate, coord(x), coord(y), coord(z)
hidden_size = 32
num_layers = 5
output_size = 1
task_type = 'extrap'
param_type = 'layers'
data = np.load(str(perc)+'Percent.npz')
coords = np.array([data["x"].tolist(), data["y"].tolist(), data["z"].tolist()]).T
connectivity = data["connectivity"]


Exporting original WSS

In [11]:
index = 'WSS'
output_directory='_'.join(['output', task_type, index])
os.makedirs(output_directory, exist_ok=True)
save_folder = '_'.join(['export',index+'/'])
data_type = 'original' #prediction, original
y = torch.load(save_folder+'_'.join(['original', 'data', str(perc)]))

time_steps = data[index].shape[0]
for t in range(time_steps):
    h5_filename = f'{data_type}_{t:04d}.h5'
    xdmf_filename = f'{data_type}_{t:04d}.xdmf'
    
    # Create HDF5 file for the current time step.
    with h5py.File(output_directory + "/" + h5_filename, 'w') as f:
        f.create_dataset('coordinates', data=coords)
        f.create_dataset('connectivity', data=connectivity)
        f.create_dataset(index, data=y[t,:])

    # Generate XDMF file for the current time step.
    xdmf_content = f"""<?xml version="1.0" ?>
<Xdmf Version="3.0">
  <Domain>
    <Grid Name="Time step {t}" GridType="Uniform">
      <Time Value="{t}" />
      <Geometry GeometryType="XYZ">
        <DataItem Dimensions="{coords.shape[0]} {coords.shape[1]}" NumberType="Float" Precision="8" Format="HDF">
          {h5_filename}:/coordinates
        </DataItem>
      </Geometry>
      <Topology TopologyType="Triangle" NumberOfElements="{connectivity.shape[0]}">
        <DataItem Dimensions="{connectivity.shape[0]} {connectivity.shape[1]}" NumberType="UInt" Format="HDF">
          {h5_filename}:/connectivity
        </DataItem>
      </Topology>
      <Attribute Name="WSS" AttributeType="Scalar" Center="Node">
        <DataItem Dimensions="{coords.shape[0]}" NumberType="Float" Precision="8" Format="HDF">
          {h5_filename}:/WSS
        </DataItem>
      </Attribute>
    </Grid>
  </Domain>
</Xdmf>
"""
    # Write the XDMF file.
    with open(output_directory + "/" + xdmf_filename, "w") as f:
        f.write(xdmf_content)


Exporting WSS prediction

In [27]:
index = 'WSS'
output_directory='_'.join(['output', task_type,index])
os.makedirs(output_directory, exist_ok=True)
save_folder = '_'.join(['export',index+'/'])
data_type = 'prediction' #prediction, original
savefile_name = [task_type, param_type, str(num_layers), str(hidden_size), str(perc)+'.pt']
#savefile_name = '_'.join(['pred',index,str(num_layers),str(it)+'.pt'])
pred = torch.load(save_folder+ '_'.join(['prediction'] + savefile_name))

time_steps = data[index].shape[0]
for t in range(time_steps):
    h5_filename = f'{data_type}_{t:04d}.h5'
    xdmf_filename = f'{data_type}_{t:04d}.xdmf'
    
    # Create HDF5 file for the current time step.
    with h5py.File(output_directory + "/" + h5_filename, 'w') as f:
        f.create_dataset('coordinates', data=coords)
        f.create_dataset('connectivity', data=connectivity)
        f.create_dataset(index, data=pred[t,:])

    # Generate XDMF file for the current time step.
    xdmf_content = f"""<?xml version="1.0" ?>
<Xdmf Version="3.0">
  <Domain>
    <Grid Name="Time step {t}" GridType="Uniform">
      <Time Value="{t}" />
      <Geometry GeometryType="XYZ">
        <DataItem Dimensions="{coords.shape[0]} {coords.shape[1]}" NumberType="Float" Precision="8" Format="HDF">
          {h5_filename}:/coordinates
        </DataItem>
      </Geometry>
      <Topology TopologyType="Triangle" NumberOfElements="{connectivity.shape[0]}">
        <DataItem Dimensions="{connectivity.shape[0]} {connectivity.shape[1]}" NumberType="UInt" Format="HDF">
          {h5_filename}:/connectivity
        </DataItem>
      </Topology>
      <Attribute Name="WSS" AttributeType="Scalar" Center="Node">
        <DataItem Dimensions="{coords.shape[0]}" NumberType="Float" Precision="8" Format="HDF">
          {h5_filename}:/WSS
        </DataItem>
      </Attribute>
    </Grid>
  </Domain>
</Xdmf>
"""
    # Write the XDMF file.
    with open(output_directory + "/" + xdmf_filename, "w") as f:
        f.write(xdmf_content)


In [3]:
index= 'OSI'
output_directory='output_'+index
os.makedirs(output_directory, exist_ok=True)
save_folder = '_'.join(['export',index+'/'])
data_type = 'original' #prediction, original
savefile_name = '_'.join([data_type, 'data', str(perc)+'.pt'])
pred = torch.load(save_folder+savefile_name)
time_steps = data[index].shape[0]
for t in range(time_steps):
    h5_filename = f'{data_type}_{t:04d}.h5'
    xdmf_filename = f'{data_type}_{t:04d}.xdmf'
    
    # Create HDF5 file for the current time step.
    with h5py.File(output_directory + "/" + h5_filename, 'w') as f:
        f.create_dataset('coordinates', data=coords)
        f.create_dataset('connectivity', data=connectivity)
        f.create_dataset('OSI', data=pred[t,:])

    # Generate XDMF file for the current time step.
    xdmf_content = f"""<?xml version="1.0" ?>
<Xdmf Version="3.0">
  <Domain>
    <Grid Name="Time step {t}" GridType="Uniform">
      <Time Value="{t}" />
      <Geometry GeometryType="XYZ">
        <DataItem Dimensions="{coords.shape[0]} {coords.shape[1]}" NumberType="Float" Precision="8" Format="HDF">
          {h5_filename}:/coordinates
        </DataItem>
      </Geometry>
      <Topology TopologyType="Triangle" NumberOfElements="{connectivity.shape[0]}">
        <DataItem Dimensions="{connectivity.shape[0]} {connectivity.shape[1]}" NumberType="UInt" Format="HDF">
          {h5_filename}:/connectivity
        </DataItem>
      </Topology>
      <Attribute Name="OSI" AttributeType="Scalar" Center="Node">
        <DataItem Dimensions="{coords.shape[0]}" NumberType="Float" Precision="8" Format="HDF">
          {h5_filename}:/OSI
        </DataItem>
      </Attribute>
    </Grid>
  </Domain>
</Xdmf>
"""
    # Write the XDMF file.
    with open(output_directory + "/" + xdmf_filename, "w") as f:
        f.write(xdmf_content)

TAWSS export

In [7]:
TAWSS = torch.load('export_TAWSS/TAWSS.pt')
TAWSSp = torch.load('export_TAWSS/TAWSS_pred.pt')
diff = np.abs(TAWSS-TAWSSp)/np.abs(TAWSS)
data_type = 'TAWSS_rel_error'

pred = diff

In [8]:
h5_filename = f'{data_type}.h5'
xdmf_filename = f'{data_type}.xdmf'

# Create HDF5 file for the current time step.
with h5py.File(output_directory + "/" + h5_filename, 'w') as f:
    f.create_dataset('coordinates', data=coords)
    f.create_dataset('connectivity', data=connectivity)
    f.create_dataset('TAWSS_rel_error', data=pred)

# Generate XDMF file for the current time step.
xdmf_content = f"""<?xml version="1.0" ?>
<Xdmf Version="3.0">
<Domain>
<Grid Name="Time step {t}" GridType="Uniform">
  <Time Value="{t}" />
  <Geometry GeometryType="XYZ">
    <DataItem Dimensions="{coords.shape[0]} {coords.shape[1]}" NumberType="Float" Precision="8" Format="HDF">
      {h5_filename}:/coordinates
    </DataItem>
  </Geometry>
  <Topology TopologyType="Triangle" NumberOfElements="{connectivity.shape[0]}">
    <DataItem Dimensions="{connectivity.shape[0]} {connectivity.shape[1]}" NumberType="UInt" Format="HDF">
      {h5_filename}:/connectivity
    </DataItem>
  </Topology>
  <Attribute Name="TAWSS_rel_error" AttributeType="Scalar" Center="Node">
    <DataItem Dimensions="{coords.shape[0]}" NumberType="Float" Precision="8" Format="HDF">
      {h5_filename}:/TAWSS_rel_error
    </DataItem>
  </Attribute>
</Grid>
</Domain>
</Xdmf>
"""
# Write the XDMF file.
with open(output_directory + "/" + xdmf_filename, "w") as f:
    f.write(xdmf_content)